In [24]:
#install.packages("RTextTools")
#install.packages("tokenizers")
#install.packages("lime")
library(tokenizers)
library(RTextTools)
library(lime)

also installing the dependencies 'colorspace', 'farver', 'labeling', 'munsell', 'RColorBrewer', 'viridisLite', 'gtable', 'isoband', 'scales', 'ggplot2', 'assertthat', 'gower'





The downloaded binary packages are in
	/var/folders/t6/_6720_rx5cvf7sc1r110_6f80000gn/T//RtmpvVVDfE/downloaded_packages


In [165]:
data(test_sentences)
stopwords <- stop_words_sentences
test <- test_sentences
train <- train_sentences
for (palabra in stopwords) {
  test$text <- gsub(paste0("\\b", palabra, "\\b"), "", test$text)
  train$text <- gsub(paste0("\\b", palabra, "\\b"), "", train$text)
}

df_final <- rbind(train, test)
df_final$text <- as.factor(df_final$text)
df_final$class.text <- as.factor(df_final$class.text)


df_final[1194,]$class.text <- "MISC"
df_final[3061,]$class.text <- "MISC"
df_final[1187,]$class.text <- "CONT"
df_final[2701,]$class.text <- "CONT"
df_final[1180,]$class.text <- "MISC"
df_final[1189,]$class.text <- "MISC"
df_final[1193,]$class.text <- "MISC"
df_final[2828,]$class.text <- "MISC"
df_final$class.text <- as.numeric(df_final$class.text)
# CONT = 4
# MISC = 6
# AIMX = 1
# OWNX = 13
# BASE = 3



In [166]:
doc_matrix <- create_matrix(df_final$text, language="english", removeNumbers=FALSE,stemWords=FALSE, removeSparseTerms=.998) #Usa package tm para preprocesamiento.


container <- create_container(doc_matrix, df_final$class.text, trainSize=1:2517, testSize=2518:3117, virgin=FALSE)
#container <- create_container(doc_matrix, df_final$class.text, trainSize=1:200, testSize=201:300, virgin=FALSE) #divide conjunto en entrenamiento y validacion
RFmodel <- train_model(container, algorithm="RF", ntree=300)
RFresults <- classify_model(container, RFmodel)


# VIEW THE RESULTS BY CREATING ANALYTICS
RFanalytics <- create_analytics(container, RFresults)
RFanalytics@algorithm_summary #: SUMMARY OF PRECISION, RECALL, F-SCORES, AND ACCURACY SORTED BY TOPIC CODE FOR EACH ALGORITHM
RFanalytics@label_summary     #: SUMMARY OF LABEL (e.g. TOPIC) ACCURACY
RFanalytics@document_summary  #: RAW SUMMARY OF ALL DATA AND SCORING
RFanalytics@ensemble_summary  #: SUMMARY OF ENSEMBLE PRECISION/COVERAGE. USES THE n VARIABLE PASSED INTO create_analytics()

Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom functions are ignored"
Warning message in TermDocumentMatrix.SimpleCorpus(x, control):
"custom tokenizer is ignored"


,FORESTS_PRECISION,FORESTS_RECALL,FORESTS_FSCORE
,<dbl>,<dbl>,<dbl>
1,0.86,0.49,0.62
3,0.54,0.54,0.54
4,0.52,0.48,0.50
6,0.90,0.94,0.92
13,0.81,0.81,0.81


,NUM_MANUALLY_CODED,NUM_CONSENSUS_CODED,NUM_PROBABILITY_CODED,PCT_CONSENSUS_CODED,PCT_PROBABILITY_CODED,PCT_CORRECTLY_CODED_CONSENSUS,PCT_CORRECTLY_CODED_PROBABILITY
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,41,22,22,53.65854,53.65854,46.34146,46.34146
2,0,2,2,Inf,Inf,NaN,NaN
3,13,13,13,100.00000,100.00000,53.84615,53.84615
4,25,23,23,92.00000,92.00000,48.00000,48.00000
6,366,382,382,104.37158,104.37158,93.44262,93.44262
9,0,0,0,NaN,NaN,NaN,NaN
10,0,1,1,Inf,Inf,NaN,NaN
11,0,0,0,NaN,NaN,NaN,NaN
12,0,1,1,Inf,Inf,NaN,NaN


,FORESTS_LABEL,FORESTS_PROB,MANUAL_CODE,CONSENSUS_CODE,CONSENSUS_AGREE,CONSENSUS_INCORRECT,PROBABILITY_CODE,PROBABILITY_INCORRECT
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,13,0.99,13,13,1,0,13,0
2,6,0.76,6,6,1,0,6,0
3,6,0.95,6,6,1,0,6,0
4,13,0.61,13,13,1,0,13,0
5,1,0.42,13,1,1,1,1,1
6,6,1.00,6,6,1,0,6,0
7,6,0.99,6,6,1,0,6,0
8,6,0.92,6,6,1,0,6,0
9,6,0.44,6,6,1,0,6,0


,n-ENSEMBLE COVERAGE,n-ENSEMBLE RECALL
n >= 1,1,0.84
